$\Huge{\textbf{Clouds Course}}$

$\small{\textbf{Assignment 1 Deliverable}}$

---

$\color{blue}\textbf{Name :}$

$\boxed{\text{        Brice Robert          }}$

$\color{blue}\textbf{Email :}$

$\boxed{\text{robert@eurecom.fr}}$

---

# Create a free Azure account

$\color{blue}\textbf{Did you run into any problems with creating an Azure account?}$

---

$\color{blue} \textbf{Your answer:}$ Having to deal with the personal account before hand was confusing but finally the Azure Account was credited using the EURECOM’s email address.

# Deploy a website on Virtual Machine

$\color{blue}\textbf{What is the VM instance type you used? How many cores and memory does it have?}$

--- 

$\color{blue} \textbf{Your answer:}$ 

```bash
VM_SIZE=$(az vm show --name lab1-vm \
    --resource-group lab1-resources --query "hardwareProfile.vmSize" -o tsv)
az vm list-sizes --location east-us --query "[?name=='$VM_SIZE']" -o table
```
> Returns
```powershell
MaxDataDiskCount  MemoryInMB  Name           NumberOfCores  OsDiskSizeInMB  ResourceDiskSizeInMB
----------------  ----------  -------------  -------------  --------------  ----------------------
2                 512         Standard_B1ls  1              1047552         4096
```
-	Name: Standard_B1ls (The VM instance type).
-	NumberOfCores: 1 (This instance has 1 core).
-	MemoryInMB: 512 MB (The VM has 512 MB of RAM).


--- 

$\color{blue}\textbf{What is the size of the virtual disk used by the VM? What type is it?}$

--- 

$\color{blue} \textbf{Your answer:}$ 

```bash
az vm show --name lab1-vm --resource-group lab1-resources \
--query "storageProfile.osDisk.{Size:diskSizeGb, Type:managedDisk.storageAccountType}" -o table
```
> Returns
```powershell
Size    Type
------  ------------
30      Standard_LRS
```

-	Size: The virtual disk size is defined by the diskSizeGb field (e.g., 30 GB).
- Type: The storage type is defined by the storageAccountType field (e.g., Standard_LRS).

--- 

$\color{blue}\textbf{What is the public ip address of the VM where we can access your site?}$

--- 

$\color{blue} \textbf{Your answer:}$ 

```
az vm list-ip-addresses --resource-group lab1-resources --output table
```
> Returns
```powershell
VirtualMachine    PublicIPAddresses    PrivateIPAddresses
----------------  -------------------  --------------------
lab1-vm           52.168.95.147        10.0.1.4
```
#### Public IP Address:

 http://52.168.95.147


# Provision disks in VM and measure IOPS

$\color{blue}\textbf{What VM type did you resize to?}$ 

$\color{blue}\textbf{What is the difference in spec between this type and the one you used before?}$

--- 

$\color{blue} \textbf{Your answer:}$ 

```
VM Type: Standard_DS3_v2
```

| **Specification**           | **Standard_DS3_v2**          | **Standard_B1ls**           |
|-----------------------------|-----------------------------|-----------------------------|
| **vCPUs**                   | 4                           | 1                           |
| **Memory**                  | 14 GiB                     | 0.5 GiB                     |
| **Temp Storage (SSD)**      | 28 GiB                     | 4 GiB                       |
| **Max Data Disks**          | 16                         | 2                           |
| **Max IOPS (Cached/Uncached)** | 16,000 / 12,800           | 200 / 320                   |
| **Max Network Interfaces**  | 4                          | 2                           |
| **Accelerated Networking**  | Supported                  | Not Supported               |
| **Use Case**                | Enterprise apps, DBs       | Low-utilization, lightweight apps |

**Summary**:  
The **DS3_v2** is a high-performance VM for demanding workloads, while the **B1ls** is a budget-friendly option for lightweight tasks.



--- 

$\color{blue}\textbf{What type of disks did you add? What were their sizes?}$ 

$\color{blue}\textbf{According to Azure, what is their expected sequencing bandwidth and random IOPS?}$

--- 

$\color{blue} \textbf{Your answer:}$ 

Type of disks and sizes:

- **Standard SSD**: 50 GiB, `/dev/sdb` (`StandardSSD_LRS`)  
- **Premium SSD**: 50 GiB, `/dev/sdc` (`Premium_LRS`)

The virtual machine 'lab1-vm' has two attached data disks:

| Lun | Name               | Caching | CreateOption | DiskSizeGb | ToBeDetached | DeleteOption |
|-----|--------------------|---------|--------------|------------|--------------|--------------|
| 0   | standard-ssd-disk  | ReadOnly| Attach       | 50         | False        | Detach       |
| 1   | premium-ssd-disk   | ReadOnly| Attach       | 50         | False        | Detach       |

Where:

- `/dev/sdb` corresponds to 'standard-ssd-disk' (Standard SSD).
- `/dev/sdc` corresponds to 'premium-ssd-disk' (Premium SSD).

According to Azure's documentation, the expected performance metrics for these disk types are:

**Standard SSD (E10 - 50 GiB):**
- **IOPS:** 500
- **Throughput:** 60 MB/s

**Premium SSD (P10 - 50 GiB):**
- **IOPS:** 500
- **Throughput:** 100 MB/s

These specifications are detailed in Azure's managed disk pricing documentation. 

Comparing these expected values to the `fio` test results:

- **`/dev/sdb` (Standard SSD):**
  - Measured IOPS: 3,128
  - Measured Throughput: 196 MiB/s (approximately 205 MB/s)

- **`/dev/sdc` (Premium SSD):**
  - Measured IOPS: 421
  - Measured Throughput: 26.3 MiB/s (approximately 27.6 MB/s)

The measured performance for `/dev/sdb` significantly exceeds Azure's documented expectations for a Standard SSD of this size, while `/dev/sdc` falls below the expected performance for a Premium SSD.

I checked if there was a mismatch between the 2 disks but couldn't find any.

--- 

$\color{blue}\textbf{What is the sequencing bandwidth and random IOPS you measured with FIO?}$

---

$\color{blue} \textbf{Your answer:}$ 

- **Standard SSD (`/dev/sdb`)**:  
  - **Sequencing Bandwidth**: 196 MiB/s  
  - **Random IOPS**: 3,128  

- **Premium SSD (`/dev/sdc`)**:  
  - **Sequencing Bandwidth**: 26.3 MiB/s  
  - **Random IOPS**: 421  

Here’s a detailed explanation of the **FIO test results** for the two disks:

### **Standard SSD (`/dev/sdb`):**
1. **Sequencing Bandwidth (196 MiB/s):**  
   - This indicates the disk's performance when reading/writing large blocks of data sequentially. 
   - The measured value of **196 MiB/s** is typical for a Standard SSD, as sequential operations leverage the SSD's design for high throughput.

2. **Random IOPS (3,128):**  
   - **IOPS (Input/Output Operations Per Second)** measures the performance of small, random read/write operations (4 KB blocks). 
   - A higher IOPS value reflects the disk's ability to handle small, frequent operations efficiently.
   - The value of **3,128 IOPS** aligns with the Standard SSD's performance for workloads with random access patterns (e.g., database queries).

---

### **Premium SSD (`/dev/sdc`):**
1. **Sequencing Bandwidth (26.3 MiB/s):**  
   - Surprisingly lower than expected for a Premium SSD. This might indicate:
     - Misconfiguration in the disk or VM.
     - Bottlenecks such as network bandwidth limits (if the disk is remotely managed).

2. **Random IOPS (421):**  
   - The random IOPS measured is significantly below Azure's advertised range for Premium SSDs, which typically exceed 5,000 IOPS for this size. This low value may again reflect:
     - Incorrect caching or mounting settings.
     - Throttling due to VM configuration or tier mismatch.

---

### **Possible Discrepancies to Investigate:**
- Confirm the disk types using Azure CLI or other tools (`az disk show`).
- Ensure the disks are attached with proper configurations (e.g., Premium SSDs benefit from `ReadWrite` caching).
- Review the VM size to confirm it supports the performance expected for these disk types.
- Rerun tests with adjusted parameters, such as `numjobs`, `iodepth`, and block sizes (`bs`) to ensure realistic workloads.

#### All the FIO testing results can be seen here:


- https://github.com/setrar/Clouds/tree/main/Labs/Azure/lab1/IaC#fio

# Deploy dockerized static site via Azure ACR

$\color{blue}\textbf{What is the public ip address of the ACI }$ 

$\color{blue}\textbf{deployed container instance where we can access your site?}$

---

$\color{blue} \textbf{Your answer:}$ 

http://acrclouds2025eurbr.eastus.azurecontainer.io

#### All the manual commands needed to manage the ACR can be found here

https://github.com/setrar/Clouds/blob/main/Labs/Azure/lab1/IaC/README.md#docker

---

$\color{blue}\textbf{What is the size of the container image used?}$ 

---

$\color{blue} \textbf{Your answer:}$ 


$\color{blue}\textbf{ (You can find this by listing the size of all images in your container registry)}$

```
docker image ls
```
> Returns:
```powershell
REPOSITORY                                  TAG       IMAGE ID       CREATED             SIZE
acrclouds2025eurbr.azurecr.io/static-site   v1        e62dc83210d6   About an hour ago   192MB
static-site                                 latest    e62dc83210d6   About an hour ago   192MB
nginx                                       latest    9bea9f2796e2   7 weeks ago         192MB
```

Docker Image size on the VM: 192MB


```
az acr repository show-manifests --name acrclouds2025eurbr \
   --repository static-site --detail --query '[].{Image: tags[0], Size: imageSize}' --output table
```
> Returns
```powershell
This command has been deprecated and will be removed in a future release. Use 'acr manifest list-metadata' instead.
Image    Size
-------  --------
v1       72972941
```

Container Size on ACR: 69MB

$\text{Size in MB} = \frac{72972941}{1048576} \approx 69.58 \, \text{MB}$

So, $72972941 \, \text{bytes} \approx 69 \, \text{MB}$ (rounded to the nearest whole number).

Which is $\frac{1}{2}$ of the original docker image size


# Create Azure webapp static site

$\color{blue}\textbf{What is the public address of the webapps static site?}$

---

$\color{blue} \textbf{Your answer:}$ 

https://webappclouds2025eurbr.azurewebsites.net

#### The IaC file that created the WebApp can be found here

- [ ] webapps.tf (.tf for terraform)

https://github.com/setrar/Clouds/blob/main/Labs/Azure/lab1/IaC/webapps.tf

- [ ] The github repo containing the APS.NET content can be found here 

https://github.com/setrar/CloudsASPXContent


# Deploy static site on Azure Blob Store

$\color{blue}\textbf{What is the public address of the Blob store static site?}$

---

$\color{blue} \textbf{Your answer:}$ 

https://blobstoreclouds2025eurbr.z13.web.core.windows.net

#### The IaC file that created the blob store can be found here

- [ ] blockstores.tf (.tf for terraform)

https://github.com/setrar/Clouds/blob/main/Labs/Azure/lab1/IaC/blockstores.tf


---

$\Large{\textbf{Summary Questions. }}$

$\color{blue}\textbf{You should have created a RG for each assignment separately.}$ 
$\color{blue}\textbf{Report the cost distribution across various RGs that you can find in }$ 
$\color{blue}\textbf{Azure Cost Analysis under Cost Management.}$

---

$\color{blue} \textbf{Your answer:}$ 

For the entire lab (Assignement 1), one resource group was used `lab1-resources` 

€2.20

IaC (Infrastructure as Code), using `OpenTofu` helps saving cost by creating all the infrastructure at once and tearing it down all together when needed. Some extra commands needed to applied manually but in general running the scripts took about 1 minute to launch.

![image](images/lab1-cost-analysis.png)